### Generation of Synthetic Showers and its analysis  

Let's generate EM showers with GANs, which has **already** trained

### Prerequistes 

1. Download :
   * **training** data: `/afs/desy.de/user/e/eren/public/gamma.hdf5`
   * **weights** of trained neural network : `/afs/desy.de/user/e/eren/public/params_generator.hdf5`
2. Make sure you installed keras : $ `conda install -c anaconda keras`
3. Install scikit-learn : `pip install scikit-learn`
4. Checkout the git repo : `https://github.com/EnginEren/CaloGAN.git` 
5. Install h5py : `pip install h5py`



In [2]:
import keras
from keras.layers import Input, Lambda, Activation, AveragePooling2D, UpSampling2D
from keras.models import Model
from keras.layers.merge import multiply
import keras.backend as K

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib import ticker, cm
from matplotlib.colors import LogNorm

Using TensorFlow backend.


### load training data

In [3]:
import h5py
g = h5py.File('path/to/gamma.hdf5', 'r')

## Loading Geant4 images
gamma = []
for j in range(5): 
    gamma.append(g['layer_{}'.format(j)][:75000])


vmin = [ gamma[k].mean(axis=0).min() for k in range(5)]
vmax = [ gamma[k].mean(axis=0).max() for k in range(5) ]

sizes = [gamma[0].shape[1], gamma[0].shape[2]] * 5

real_images_gamma = [gamma[k] for k in range(5)]


### Start generation

In [ ]:
latent_size = 1024
%cd /home/eren/CaloGAN/models/   ## change it to your path
from architectures import build_generator, build_discriminator, sparse_softmax
from ops import scale, inpainting_attention
%cd -

In [ ]:
# input placeholders
latent = Input(shape=(latent_size, ), name='z') # noise
input_energy = Input(shape=(1, ), dtype='float32') # requested energy of the particle shower
generator_inputs = [latent, input_energy]

# multiply the (scaled) energy into the latent space
h = Lambda(lambda x: x[0] * x[1])([latent, scale(input_energy, 100)])

# build 5 LAGAN-style generators (checkout out `build_generator` in architectures.py)
img_layer = []
for i in range(5):
    img_layer.append(build_generator(h, 12, 12))
        
# inpainting
zero2one = AveragePooling2D(pool_size=(1, 1))(UpSampling2D(size=(1, 1))(img_layer[0]))
img_layer[1] = inpainting_attention(img_layer[1], zero2one)
for j in range(1,4):
    one2N = AveragePooling2D(pool_size=(1, 1))(img_layer[j])
    img_layer[j+1] = inpainting_attention(img_layer[j+1], one2N)

generator_outputs = []

## outputs
for k in range(5):
    generator_outputs.append(Activation('relu')(img_layer[k]))
        

Now that the net is instantiated, we can use it to generate images. To do so, we need to provide inputs. In particular, the latent space expects some normally distributed noise, while the requested energy can be an array of your choice.

In [ ]:
# build the actual model
generator = Model(generator_inputs, generator_outputs)
# load trained weights
generator.load_weights('path/to/weight')
## noise
noise = np.random.normal(0, 1, (1000, latent_size))
## We would like to generate 50 GeV showers 
sampled_energy = np.random.uniform(50, 50, (1000, 1))


Our net was trained with an uniform spectrum of energies between 10 and 100 GeV, so it should be safe to request any list of energies in that range. You could also try to request an energy outside of that range and see what happens.

In [ ]:
images = generator.predict([noise, sampled_energy], verbose=True)

In [ ]:
# we need to multiply images by 1000 to match order of magnitude of generated images to real images
## watch out for python3 changes: map method doesnt return list!
images = list(map(lambda x: np.squeeze(x * 1000), images))

### Task that would be usefull for the project:

1. Generate 25, 50, 75, 100, 125, 150, 175, 200 GeV showers (10000 for each)
2. Calculate $\sigma_E / E$ as a function of $E_{vis}$:
    * for G4
    * for GAN
    
3. And plot them